# Road Sign Recognition 


# Table of contents
1. [Introduction](#introduction)
2. [Problem](#Problem)
3. [Methods](#Methods)






## Introduction <a name="introduction"></a>


Artificial Intelligence (AI) has emerged as a powerful tool for solving complex problems and transforming various industries. Its ability to process large volumes of data, learn from patterns, and make intelligent decisions has made it a valuable asset in problem-solving scenarios. AI offers several advantages that enable the resolution of problems in a simpler and more efficient manner. One of the key strengths of AI is its capacity to handle and analyze massive amounts of data quickly and accurately. Traditional problem-solving approaches often struggle with large datasets or complex patterns that are difficult for humans to comprehend. AI algorithms, on the other hand, can effortlessly process and extract valuable insights from vast data sets, enabling the identification of hidden patterns or correlations that might not be evident to human observers. This capability allows AI to tackle complex problems with ease.

## Problem <a name="Problem"></a>


The accurate detection and interpretation of road signs play a vital role in ensuring road safety and efficient traffic management. However, traditional methods of road sign recognition often face challenges in handling the diverse range of signs, varying environmental conditions, and real-time processing requirements. This necessitates the exploration of AI-based solutions to overcome these limitations and enhance road sign recognition capabilities. AI-powered road sign recognition systems can effectively address the complexity and variability of road signs. The problem lies in the diverse shapes, colors, sizes, and designs of road signs, making it challenging for traditional algorithms to accurately detect and classify them. AI, specifically deep learning algorithms, can learn intricate patterns and features from large datasets, enabling the development of robust models capable of accurately identifying and classifying various types of road signs.


## Methods <a name="Methods"></a>

In the realm of road sign recognition, advancements in computer vision techniques have paved the way for effective detection and classification methods. One such approach involves the utilization of features like Hue, Histogram of Oriented Gradients (HOG), and Haar-like features, coupled with Support Vector Machine (SVM) classification and the random forest algorthm.

Hue is a color attribute that represents the dominant color in an image. By extracting the hue component from road sign images, we can leverage its distinctive characteristics to differentiate between different types of signs. This feature provides valuable information for the classification process, aiding in accurate recognition.

The Histogram of Oriented Gradients (HOG) technique captures the distribution of gradient orientations within an image. By analyzing the intensity gradients, HOG can effectively capture the shape and edges of road signs. These features serve as strong discriminative cues, enabling the system to identify and classify signs based on their unique shapes and contours.

Additionally, Haar-like features offer an efficient approach to analyze patterns within an image. These features involve comparing adjacent rectangular regions to detect specific visual patterns, such as edges, corners, or color contrasts. By applying Haar-like features to road sign images, the system can identify characteristic patterns that are indicative of different types of signs.

To classify the extracted features, Support Vector Machines (SVM) are commonly employed. SVM is a powerful machine learning algorithm that can efficiently categorize data points based on their features. By training an SVM model with labeled road sign images and their corresponding extracted features, the system can learn to accurately classify unseen road signs based on the learned patterns and feature representations.

The Random Forest works by analyzing each road sign based on its features (such as shape, color, symbols), and then making a prediction to classify the sign. By combining the decisions of multiple decision trees, the Random Forest achieves higher accuracy in road sign detection and is robust to noise or other disturbances in the images.

First of all we need to download the data. We'll use the GTSRB Training Images Set as our whole Dataset and divide it later to a training and test set. In order to get this Notebook working, you need to do the following steps:
- Change the referencing folderpath to yours
- install opencv via the terminal

The Following tasks numbered in the cells (1., 2., 3., ...) are shown in the Table of Contents:

1. Importing Librarys
2. Downloading the Data
3. Renaming the Data
4. Extracting the HOG Features
5. Split the Data and Train the model and evaluating - HOG -Random Forest
6. Split the Data and Train the model and evaluating - HOG -Support Vector Machine
7. Extracting the HUE Features
8. Split the Data and Train the model and evaluating - HUE -Random Forest
9. Split the Data and Train the model and evaluating - HUE -Support Vector Machine


In [1]:
# 1.Importing Librarys
import requests
import zipfile
import os
import numpy as np
from skimage.io import imshow, imread
from skimage.feature import hog
from skimage import exposure
import skimage.transform as transform
import skimage.io
import skimage.exposure
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.datasets import make_classification
import cv2 # pls enter this command in your Terminal: pip3 install opencv-python

In [1]:
# 2.Downloading the Data
# URL of the dataset
url = "https://sid.erda.dk/public/archives/daaeac0d7ce1152aea9b61d9f1e19370/GTSRB_Final_Training_Images.zip"

# Send a GET request to the URL
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Save the downloaded file
    with open("dataset.zip", "wb") as file:
        file.write(response.content)
    
    # Extract the contents of the ZIP file
    with zipfile.ZipFile("dataset.zip", "r") as zip_ref:
        zip_ref.extractall("dataset_folder_train_images")
    
    print("Dataset downloaded and extracted successfully.")
else:
    print("Failed to download the dataset.")

Dataset downloaded and extracted successfully.


In [4]:
# 3.Renaming the Data
# Renaming the datainfrastructure
def rename_folders(folder_path):
    for folder_name in os.listdir(folder_path):
        folder_old_name = os.path.join(folder_path, folder_name)
        folder_new_name = os.path.join(folder_path, str(int(folder_name)))
        
        # Rename the folder
        os.rename(folder_old_name, folder_new_name)
        
# Be sure, to insert the right folderpath here!  
rename_folders('/home/jovyan/ML Projekkkkt/App.ML/dataset_folder_train_images/GTSRB/Final_Training/Images/')

In [2]:
# 4.Extracting the HOG Features
#read the dataset and put it in an array for HOG_features:
def read_data_HOG(folder_path):
    X = []
    y = []

    for class_label in range(43):
        class_folder = os.path.join(folder_path, str(class_label))
        
        for file_name in os.listdir(class_folder):
            if file_name.endswith(".ppm"):
                file_path = os.path.join(class_folder, file_name)
                
                # Code snippet for hogz
                img = skimage.io.imread(file_path, plugin='matplotlib')
                img_gray = skimage.color.rgb2gray(img)
                reshaped_img = transform.resize(img_gray, (40, 40))
                normalized_image = skimage.exposure.rescale_intensity(reshaped_img, in_range='image', out_range=(0, 1))
                hog_features, hog_image = hog(normalized_image, orientations=8, pixels_per_cell=(6, 6), cells_per_block=(2, 2), visualize=True)
                hog_image_rescaled = exposure.rescale_intensity(hog_image, in_range=(0, 5))
                hog_features_array = hog_features.reshape(-1)
                X.append(hog_features_array)
                y.append(class_label)

    return np.array(X), np.array(y)

folder_path_train = "/home/jovyan/ML Projekkkkt/App.ML/dataset_folder_train_images/GTSRB/Final_Training/Images"

X_hog,y_hog = read_data_HOG(folder_path_train)

In [3]:
# 5.Split the Data and Train the model and evaluating - HOG - Random Forest
#random Forest for HOG_Features
X_train_forest_hog, X_test_forest_hog, y_train_forest_hog, y_test_forest_hog = train_test_split(X_hog, y_hog, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train_scaled_forest_hog = scaler.fit_transform(X_train_forest_hog)
X_test_scaled_forest_hog = scaler.transform(X_test_forest_hog)

# Create and train the Random Forest model
random_forest_hog = RandomForestClassifier()
random_forest_hog.fit(X_train_scaled_forest_hog, y_train_forest_hog)

# Make predictions on the test set
y_pred_forest_hog = random_forest_hog.predict(X_test_scaled_forest_hog)

# Evaluate the model
accuracy_forest_hog = accuracy_score(y_test_forest_hog, y_pred_forest_hog)
print("Accuracy for Random Forest:", accuracy_forest_hog)
print()

# Calculating precision, recall, and F1-score for each class
precision_forest_hog = precision_score(y_test_forest_hog, y_pred_forest_hog, average=None)
recall_forest_hog = recall_score(y_test_forest_hog, y_pred_forest_hog, average=None)
f1_forest_hog = f1_score(y_test_forest_hog, y_pred_forest_hog, average=None)

# Printing the evaluation metrics for each class
for class_label, prec, rec, f1_score_rf1 in zip(np.unique(y_hog), precision_forest_hog, recall_forest_hog, f1_forest_hog):
    print(f"Class: {class_label}")
    print(f"Precision: {prec}")
    print(f"Recall: {rec}")
    print(f"F1-Score: {f1_score_rf1}")
    print()

Accuracy for Random Forest: 0.9422341239479725

Class: 0
Precision: 1.0
Recall: 0.5526315789473685
F1-Score: 0.711864406779661

Class: 1
Precision: 0.8961303462321792
Recall: 0.8870967741935484
F1-Score: 0.8915906788247213

Class: 2
Precision: 0.8329853862212944
Recall: 0.8866666666666667
F1-Score: 0.8589881593110872

Class: 3
Precision: 0.9375
Recall: 0.9107142857142857
F1-Score: 0.9239130434782609

Class: 4
Precision: 0.9140271493212669
Recall: 0.9665071770334929
F1-Score: 0.9395348837209303

Class: 5
Precision: 0.7864864864864864
Recall: 0.7994505494505495
F1-Score: 0.7929155313351498

Class: 6
Precision: 1.0
Recall: 1.0
F1-Score: 1.0

Class: 7
Precision: 0.8823529411764706
Recall: 0.9172661870503597
F1-Score: 0.8994708994708994

Class: 8
Precision: 0.9084249084249084
Recall: 0.8239202657807309
F1-Score: 0.8641114982578397

Class: 9
Precision: 0.981549815498155
Recall: 0.9925373134328358
F1-Score: 0.9870129870129871

Class: 10
Precision: 0.9631578947368421
Recall: 0.9891891891891892

In [4]:
# 6.Split the Data and Train the model and evaluating - HOG - Support Vector Machine
#SVM for HOG_Features
X_train_svm_hog, X_test_svm_hog, y_train_svm_hog, y_test_svm_hog = train_test_split(X_hog, y_hog, test_size=0.2, random_state=42)

clf_hog= make_pipeline(StandardScaler(),SVC(kernel="rbf", gamma= "auto"))
clf_hog.fit(X_train_svm_hog,y_train_svm_hog)
y_pred_svm_hog = clf_hog.predict(X_test_svm_hog)
accuracy_svm_hog = accuracy_score(y_test_svm_hog, y_pred_svm_hog)
print("Accuracy for a SVM:", accuracy_svm_hog)
print()

# Calculating precision, recall, and F1-score for each class
precision_svm_hog = precision_score(y_test_svm_hog, y_pred_svm_hog, average=None)
recall_svm_hog = recall_score(y_test_svm_hog, y_pred_svm_hog, average=None)
f1_svm_hog = f1_score(y_test_svm_hog, y_pred_svm_hog, average=None)

# Printing the evaluation metrics for each class
for class_label, prec, rec, f1_score_svm1  in zip(np.unique(y_hog), precision_svm_hog, recall_svm_hog, f1_svm_hog):
    print(f"Class: {class_label}")
    print(f"Precision: {prec}")
    print(f"Recall: {rec}")
    print(f"F1-Score: {f1_score_svm1}")
    print()

Accuracy for a SVM: 0.9802346340219332



TypeError: 'numpy.float64' object is not callable

In [1]:
# 7.Extracting the HUE Features
def read_data_HUE(folder_path):
    X = []
    y = []
    
    desired_width = 29
    desired_height = 30

    for class_label in range(43):
        class_folder = os.path.join(folder_path, str(class_label))
        
        
        
        for file_name in os.listdir(class_folder):
            if file_name.endswith(".ppm"):
                file_path = os.path.join(class_folder, file_name)
                
                
            
                image = cv2.imread(file_path)
                scaled_image = cv2.resize(image, (desired_width, desired_height)) 
                image_hsv = cv2.cvtColor(scaled_image, cv2.COLOR_BGR2HSV) 
                
                border_size = 5
                image_hsv_cropped = image_hsv[border_size:-border_size, border_size:-border_size] 
                hue_values = image_hsv_cropped[:, :, 0].flatten()
                
                histogram, _ = np.histogram(hue_values, bins=256, range=[0, 256])
                normalized_histogram = histogram.astype(np.float64) / np.sum(histogram)
                dimensionality = len(normalized_histogram)
               
                
                X.append(hue_values)
                y.append(class_label)

    return np.array(X), np.array(y)

folder_path_train = "/home/jovyan/ML Projekkkkt/App.ML/dataset_folder_train_images/GTSRB/Final_Training/Images"

X_hue,y_hue = read_data_HUE(folder_path_train)

NameError: name 'os' is not defined

In [2]:
# 8.Split the Data and Train the model and evaluating - HUE - Random Forest
#random Forest for HUE features
X_train_forest_hue, X_test_forest_hue, y_train_forest_hue, y_test_forest_hue = train_test_split(X_hue, y_hue, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train_scaled_forest_hue = scaler.fit_transform(X_train_forest_hue)
X_test_scaled_forest_hue = scaler.transform(X_test_forest_hue)

# Create and train the Random Forest model
random_forest_hue = RandomForestClassifier()
random_forest_hue.fit(X_train_scaled_forest_hue, y_train_forest_hue)

# Make predictions on the test set
y_pred_forest_hue = random_forest_hue.predict(X_test_scaled_forest_hue)

# Evaluate the model
accuracy_forest_hue = accuracy_score(y_test_forest_hue, y_pred_forest_hue)
print("Accuracy for Random Forest:", accuracy_forest_hue)
print()

# Calculating precision, recall, and F1-score for each class
precision_forest_hue = precision_score(y_test_forest_hue, y_pred_forest_hue, average=None)
recall_forest_hue = recall_score(y_test_forest_hue, y_pred_forest_hue, average=None)
f1_forest_hue = f1_score(y_test_forest_hue, y_pred_forest_hue, average=None)

# Printing the evaluation metrics for each class
for class_label, prec, rec, f1_score_rf2 in zip(np.unique(y_hue), precision_forest_hue, recall_forest_hue, f1_forest_hue):
    print(f"Class: {class_label}")
    print(f"Precision: {prec}")
    print(f"Recall: {rec}")
    print(f"F1-Score: {f1_score_rf2}")
    print()

NameError: name 'train_test_split' is not defined

In [ ]:
# 9.Split the Data and Train the model and evaluating - HUE - Support Vector Machine
#SVM for HUE features
X_train_svm_hue, X_test_svm_hue, y_train_svm_hue, y_test_svm_hue = train_test_split(X_hue, y_hue, test_size=0.2, random_state=42)

clf_hue= make_pipeline(StandardScaler(),SVC(kernel="rbf", gamma= "auto"))
clf_hue.fit(X_train_svm_hue,y_train_svm_hue)
y_pred_svm_hue = clf_hue.predict(X_test_svm_hue)
accuracy_svm_hue = accuracy_score(y_test_svm_hue, y_pred_svm_hue)
print("Accuracy for a SVM:", accuracy_svm_hue)
print()

# Calculating precision, recall, and F1-score for each class
precision_svm_hue = precision_score(y_test_svm_hue, y_pred_svm_hue, average=None)
recall_svm_hue = recall_score(y_test_svm_hue, y_pred_svm_hue, average=None)
f1_svm_hue = f1_score(y_test_svm_hue, y_pred_svm_hue, average=None)

# Printing the evaluation metrics for each class
for class_label, prec, rec, f1_score_svm2 in zip(np.unique(y_hue), precision_svm_hue, recall_svm_hue, f1_svm_hue):
    print(f"Class: {class_label}")
    print(f"Precision: {prec}")
    print(f"Recall: {rec}")
    print(f"F1-Score: {f1_score_svm2}")
    print()